### This is about text classification of the Yelp Review https://pytorch.org/text/stable/datasets.html#yelpreviewfull
####  number of classes=5
####  number of  training samples=650000
####  number of test samples=50000


In [1]:
import torch
from torchtext.datasets import YelpReviewFull
import numpy as np
train_iter = YelpReviewFull(split='train')

In [2]:
next(train_iter)

(5,
 "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.")

In [3]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [4]:
tokenizer = get_tokenizer('basic_english')
train_iter = YelpReviewFull(split='train')


In [5]:
def yield_tokens(data_iter):
    for _,text in data_iter:
        yield tokenizer(text)


In [6]:
## this returns an object of type torchtext.vocab.vocab.Vocab
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])


In [7]:
vocab.set_default_index(vocab["<unk>"])

In [8]:
vocab(['her','rohit'])

[125, 447398]

In [9]:
## define the two functions to process the text and the labels
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline=lambda x: int(x)-1


In [10]:
text_pipeline('here is the an example')

[48, 14, 2, 68, 1985]

In [11]:
label_pipeline(3)

2

In [12]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = YelpReviewFull(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False,collate_fn=collate_batch)


In [13]:
tt=next(iter(dataloader))

In [14]:
tt

(tensor([4, 1, 3, 3, 0, 4, 4, 0], device='cuda:0'),
 tensor([ 1042,     1, 42282,  ...,  2850,  1240,    16], device='cuda:0'),
 tensor([  0, 112, 253, 365, 422, 660, 728, 801], device='cuda:0'))

In [15]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


In [16]:
import time

def train(dataloader):
    ##put the model in training mode
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    ##since the dataloadeer is using a collate_batch function it is giving our 3 outputs..
    ##label, text (in the form of words) and the offsets for identifying the sentence boudary
    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        ##loss calculation criteria
        loss = criterion(predicted_label, label)
        ##backprop
        loss.backward()
        ## instead of doing the batch norm it is doing a clipping function..this is mostly to save from exploding gradient issue
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    ##put the model in inference mode
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [17]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
vocab_size=len(vocab)
emsize=64
num_class=len(set([label for (label, text) in train_iter]))


model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = YelpReviewFull()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])


In [18]:
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)


As per the question we need to achieve the accuracy of 20% (as the number of the classes=5) +.50*20=30%


In [19]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 9649 batches | accuracy    0.391
| epoch   1 |  1000/ 9649 batches | accuracy    0.486
| epoch   1 |  1500/ 9649 batches | accuracy    0.509
| epoch   1 |  2000/ 9649 batches | accuracy    0.527
| epoch   1 |  2500/ 9649 batches | accuracy    0.537
| epoch   1 |  3000/ 9649 batches | accuracy    0.542
| epoch   1 |  3500/ 9649 batches | accuracy    0.549
| epoch   1 |  4000/ 9649 batches | accuracy    0.551
| epoch   1 |  4500/ 9649 batches | accuracy    0.551
| epoch   1 |  5000/ 9649 batches | accuracy    0.552
| epoch   1 |  5500/ 9649 batches | accuracy    0.556
| epoch   1 |  6000/ 9649 batches | accuracy    0.561
| epoch   1 |  6500/ 9649 batches | accuracy    0.563
| epoch   1 |  7000/ 9649 batches | accuracy    0.563
| epoch   1 |  7500/ 9649 batches | accuracy    0.565
| epoch   1 |  8000/ 9649 batches | accuracy    0.569
| epoch   1 |  8500/ 9649 batches | accuracy    0.566
| epoch   1 |  9000/ 9649 batches | accuracy    0.572
| epoch   1 |  9500/ 9649 ba

| epoch   7 |  9500/ 9649 batches | accuracy    0.609
-----------------------------------------------------------
| end of epoch   7 | time: 57.89s | valid accuracy    0.600 
-----------------------------------------------------------
| epoch   8 |   500/ 9649 batches | accuracy    0.614
| epoch   8 |  1000/ 9649 batches | accuracy    0.610
| epoch   8 |  1500/ 9649 batches | accuracy    0.610
| epoch   8 |  2000/ 9649 batches | accuracy    0.613
| epoch   8 |  2500/ 9649 batches | accuracy    0.609
| epoch   8 |  3000/ 9649 batches | accuracy    0.611
| epoch   8 |  3500/ 9649 batches | accuracy    0.609
| epoch   8 |  4000/ 9649 batches | accuracy    0.606
| epoch   8 |  4500/ 9649 batches | accuracy    0.610
| epoch   8 |  5000/ 9649 batches | accuracy    0.612
| epoch   8 |  5500/ 9649 batches | accuracy    0.608
| epoch   8 |  6000/ 9649 batches | accuracy    0.604
| epoch   8 |  6500/ 9649 batches | accuracy    0.611
| epoch   8 |  7000/ 9649 batches | accuracy    0.609
| epoch  

In [21]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.599


### Here we are building the data which can be used to determine the Accuracy, Precision, Recall & F1 score
This is a multiclass classification

In [25]:
model.eval()
predictedLabels=[]
trueLabels=[]
with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(test_dataloader):
        predicted_label = model(text, offsets)
        predictedLabels.append(predicted_label.argmax(1))
        trueLabels.append(label)
        

In [28]:
## we have got a list of batches of results so merging them together in a single list

import itertools
flatTrueLabels=list(itertools.chain.from_iterable(trueLabels))
flatPredictedLabels=list(itertools.chain.from_iterable(predictedLabels))

#bringing everything to cpu as we are planning to use the sklearn method to calculate the metrics. 
#The data should be on the cpu
flatPredictedLabels=torch.stack(flatPredictedLabels).cpu()
flatTrueLabels=torch.stack(flatTrueLabels).cpu()

In [29]:
from sklearn.metrics import precision_recall_fscore_support
(prec,recall,f1,_)=precision_recall_fscore_support(flatTrueLabels, flatPredictedLabels, average='macro', zero_division=0)

In [30]:
print("Precision=",prec,"Recall=",recall,"F1=",f1)

Precision= 0.5971956406331864 Recall= 0.59948 F1= 0.5980405353749079
